In [1]:
import pandas as pd
import sys
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sentence_transformers import SentenceTransformer
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor

sys.path.append("../")
import src.data_utils as d_u
import src.feats_generation as f_g
import src.eval_utils as e_u

c:\Users\DungeonMaster3000\.conda\envs\avsoft_test_task\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DungeonMaster3000\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package stopwords is already up-to-date!


Сначала с помощью подбора гиперпараметров посмотрим, какое наилучшее решение мы можем получить, используя линейную регрессию с L2 регуляризацией

In [2]:
# Получим предобработанный датасет

df = d_u.get_preprocess_data()

X = df.drop(columns=["commit_message", "bugs"])
y = df.bugs

msg_embs = f_g.pretrained_model_sentence_emb(df.commit_message.values)

X = np.concatenate((X.to_numpy(), msg_embs), axis=1)

In [3]:
# конфигурация для поиска наилучшего решения

ridge_config = {
    "alpha": [0.5, 1.0, 1.5, 5.0, 10],
    "solver": ["auto", "svd", "cholesky", "sag"],
    "random_state": [42],
    "tol": [1e-2, 1e-3, 1e-4]
}


In [4]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

model_ridge = Ridge(random_state=42)
gs_ridge = GridSearchCV(model_ridge, ridge_config, cv=cv, scoring="neg_mean_squared_error", verbose=1)
search = gs_ridge.fit(X, y)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


In [5]:
# Посмотрим на лучшие параметры и лучшую оценку

print(search.best_params_)
print(search.best_score_)

{'alpha': 0.5, 'random_state': 42, 'solver': 'sag', 'tol': 0.01}
-1.3645594120400553


Лучшее качество для линейной модели с L2 регуляризацией - 1.3645594120400553

Теперь поперебираем параметры для нейронной сети из sklearn

In [6]:
nn_config = {
    "hidden_layer_sizes": [(100,), (300, 100,), (300, 100, 50)],
    "activation": ["logistic", "tanh", "relu"],
    "solver": ["sgd", "adam"],
    "learning_rate": ["constant", "invscaling", "adaptive"],
    "random_state": [42],
    "max_iter": [10000]
}

In [7]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

model_nn = MLPRegressor(random_state=42)
gs_nn = GridSearchCV(model_nn, nn_config, cv=cv, scoring="neg_mean_squared_error", verbose=1)
search = gs_nn.fit(X, y)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


In [9]:
# Посмотрим на лучшие параметры и лучшую оценку

print(search.best_params_)
print(search.best_score_)

{'activation': 'relu', 'hidden_layer_sizes': (300, 100), 'learning_rate': 'constant', 'max_iter': 10000, 'random_state': 42, 'solver': 'sgd'}
-1.0978673099828316


Лучшее качество для нейронной сети - 1.0978673099828316

Теперь попробуем подобрать параметры для градиентного бустинга

In [13]:
gb_config = {
    "loss": ["squared_error", "absolute_error", "huber"],
    "learning_rate": [0.1, 0.5, 1.0],
    "n_estimators": [100, 150, 200],
    "criterion": ["squared_error"],
    "max_depth": [3, 5, 7],
    "random_state": [42]    
}

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

model_gb = GradientBoostingRegressor(random_state=42)
gs_gb = GridSearchCV(model_gb, gb_config, cv=cv, scoring="neg_mean_squared_error", verbose=2)
search = gs_gb.fit(X, y)

In [16]:
# Посмотрим на лучшие параметры и лучшую оценку

print(search.best_params_)
print(search.best_score_)

{'criterion': 'squared_error', 'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 3, 'n_estimators': 150, 'random_state': 42}
-1.4590573214275597


Градиентный бустинг дает качество на даннной задаче значительно хуже нежели Ridge и MLP

Попробуем специализированную библиотеку для градиентного бустинга

In [3]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

model_lgb = LGBMRegressor(random_state=42)
# gs_gb = GridSearchCV(model_gb, gb_config, cv=cv, scoring="neg_mean_squared_error", verbose=2)
# search = gs_gb.fit(X, y)
print(np.mean(cross_val_score(model_lgb, X, y, cv=cv, scoring="neg_mean_squared_error")))

-1.7115090825707544


In [5]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

model_cgb = CatBoostRegressor(random_state=42, verbose=0)
# gs_gb = GridSearchCV(model_gb, gb_config, cv=cv, scoring="neg_mean_squared_error", verbose=2)
# search = gs_gb.fit(X, y)
print(np.mean(cross_val_score(model_cgb, X, y, cv=cv, scoring="neg_mean_squared_error")))

-1.4879989582292803


У базового регрессора из catboost качество получилось близкое к качеству модели градиентного бустинга из sklearn с параметрами подобранными по сетке. Попробуем применить его без предобработки категориальных признаков

In [6]:
df = pd.read_csv("../data/raw/АВСОФТ_тест_ML_приложение.csv")
df.drop(columns=["commit_hash"], inplace=True)

df["commit_date"] = pd.to_datetime(df.commit_date)

df = f_g.encode_work_days(df)
df = f_g.encode_work_hours(df)

if "commit_date" in df.columns:
    df.drop(columns=["commit_date"], inplace=True)

X = df.drop(columns=["commit_message", "bugs"])
y = df.bugs

msg_embs = f_g.pretrained_model_sentence_emb(df.commit_message.values)

X = pd.concat((X, pd.DataFrame(msg_embs)), axis=1)

In [7]:
X.sample(3)

,repository_name,commit_author,no_work_d,work_d,no_work_h,work_h,0,1,2,3,...,502,503,504,505,506,507,508,509,510,511
178,conductor,Dabe,0,1,0,1,0.016709,0.079920,0.003142,0.019246,...,0.016541,-0.001993,-0.009228,-0.052684,0.048641,-0.010819,0.015667,0.034769,-0.063120,0.067844
258,sensor,Victor,0,1,1,0,-0.082101,-0.019746,-0.009664,-0.031541,...,-0.005454,0.005846,0.029946,0.054653,-0.026029,-0.033068,-0.016885,0.029324,0.092721,-0.108200
85,conductor,Wendy,0,1,0,1,-0.015878,0.002328,-0.052344,-0.023136,...,0.012363,-0.017067,0.018917,0.030094,-0.020613,0.013868,-0.038820,-0.006034,0.010248,0.006112


In [8]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

model_cgb = CatBoostRegressor(random_state=42, verbose=0, cat_features=["repository_name", "commit_author"])
# gs_gb = GridSearchCV(model_gb, gb_config, cv=cv, scoring="neg_mean_squared_error", verbose=2)
# search = gs_gb.fit(X, y)

In [9]:
print(np.mean(cross_val_score(model_cgb, X, y, cv=cv, scoring="neg_mean_squared_error")))

-1.609263768264578


Вряд ли подбор параметров позволит приблизиться к лидеру(нейронная сеть), однако в дальнейшем стоит поэксперементировать

### Выводы

Наилучшим подходом в данный момент себя показало кодирование имени репозитория, а также деление на 3 группы по числу найденных ошибок авторов комммитов и кодирование их также с помощью One-Hot Encoder. Также была подтверждена гипотеза, что учитывая смысл(хотя бы частичный) сообщения коммита можно лучше предсказывать число найденных багов. Время коммита было преобразованно в пару полезных признаков: рабочее или не рабочее время и рабочий или выходной день недели. Я испробовал TfidfVectorizer и предобученную мультиязычную модель на основе Transformer и CNN для получения эмбеддинга предложения. Второй вариант дал гораздо более сильное уменьшение среднеквадратического отклонения. Также были испробованы TruncatedSVD и PCA, так как данных оч мало, а размер получаемого вектора признаков для объекта превышает общее количество объектов в имеющемся датасете. Однако, данные подходы не показали сколько-нибудь значительного улучшения качества работы модели.

Лучше всего себя на данный момент показала двухслойная нейронная сеть с функцийе активации Relu. Ее mse на кросс-валидации составило 1.0978673099828316. 

Дальнейшая работа:

1. Обучить двух(трех)слойную нейронную сеть с помощью фрэймворка Pytorch с использованием слоя Embeddings для категориальных переменных имя репозитория, имя автора коммита, рабочее время, рабочий день недели.

2. Попробовать применить стэкинг

3. Подумать над тем, какие еще признаки можн сгенерировать вручную на основе времени коммита и текста сообщения коммита.

4. Попробовать поподбирать параметры для моделей из специализированных библиотек для градиентного бустинга(lightgbm и catboost)
